# Importing Libraries

In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import Session
from sagemaker.inputs import TrainingInput

# Creating S3 bucket

In [9]:
bucket_name='bankappnew'
my_region=boto3.session.Session().region_name
print(my_region)

us-east-1


In [10]:
s3=boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('s3 bucket created')
except Exception as e:
    print('s3 error: ',e)

s3 bucket created


## set an output path where the trained model will be saved

In [11]:
prefix= 'xgboost-as-a-built-in-algo'
output_path='s3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://bankappnew/xgboost-as-a-built-in-algo/output


# Downloading The Dataset And Storing in S3

In [12]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


## Train Test split

In [13]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# Saving Train And Test Into Buckets

In [18]:
import os
import boto3
import pandas as pd
from sagemaker.inputs import TrainingInput

pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

# Upload the file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Define the S3 input for training
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train/train.csv'.format(bucket_name, prefix), content_type='text/csv')


In [20]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

# Upload the file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Define the S3 input for testing
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test/test.csv'.format(bucket_name, prefix), content_type='text/csv')

# Building Models Xgboot- Inbuilt Algorithm

In [22]:
import boto3
from sagemaker import image_uris

repo_version = '1.0-1'

# Automatically look for the XGBoost image URI and build an XGBoost container
container = image_uris.retrieve('xgboost', 
                                boto3.Session().region_name, 
                                version=repo_version)


In [23]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [25]:
estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # 5 GB
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600
)

In [26]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-08-04-05-40-50-654


2024-08-04 05:40:50 Starting - Starting the training job...
2024-08-04 05:41:06 Starting - Preparing the instances for training...
2024-08-04 05:41:49 Downloading - Downloading the training image......
2024-08-04 05:42:50 Training - Training image download completed. Training in progress.
2024-08-04 05:42:50 Uploading - Uploading generated training model[2024-08-04 05:42:42.202 ip-10-2-200-11.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determ

# Deploy Machine Learning Model As Endpoints

In [27]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-04-05-49-57-434
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-04-05-49-57-434
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-04-05-49-57-434


------!

## Prediction of the Test Data

In [29]:
import numpy as np
from sagemaker.serializers import CSVSerializer

# Prepare test data as a numpy array
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Set the content type and serializer for the predictor
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()

# Predict and process the results
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions, sep=',')
print(predictions_array.shape)


(12357,)


In [31]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

## confusion matrix

In [32]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 

